<a href="https://colab.research.google.com/github/arthurst38/deep_learning/blob/main/Auto_encodeurs_d%C3%A9bruiteurs_avec_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto-encodeurs débruiteurs avec Keras

## Vérification de l'utilisation de GPU

Allez dans le menu `Exécution > Modifier le type d'execution` et vérifiez que l'on est bien en Python 3 et que l'accélérateur matériel est configuré sur « GPU ».

In [ ]:
!nvidia-smi

## Import de TensorFlow et des autres librairies nécessaires

In [ ]:
import matplotlib.pyplot as plt
import numpy
import tensorflow.keras as keras

## Chargement de MNIST

Nous allons utiliser un prétraîtement légèrement différent des autres fois : étant donné que nous voulons pouvoir prédire les valeurs données en entrée en sortie (principe de l'auto-encodage), nous allons simplement projeter ces valeurs dans $[0, 1]$ au lieu de $[0, 255]$. Notez qu'habituellement nous ne faisons pas ça : nous normalisons en centrant sur zéro et en divisant par l'écart-type.

In [ ]:
(X_train, _), (X_test, y_test) = keras.datasets.mnist.load_data()
nb_classes = 10
input_dim = 28 * 28
X_train = X_train.reshape(-1, input_dim).astype('float32')
X_test = X_test.reshape(-1, input_dim).astype('float32')

# On utilise cette normalisation pour garder les pixel à 0 où ils sont
X_train = X_train / 255.0
X_test = X_test / 255.0


## Application d'un bruit  gaussien

In [ ]:
noise_factor = 0.5
X_train_noisy = X_train + numpy.random.normal(0, noise_factor, X_train.shape) 
X_test_noisy = X_test + numpy.random.normal(0, noise_factor, X_test.shape)

# On clip les valeurs pour éviter les pixels plus blanc que blanc (ou plus noir
# que noir) 
numpy.clip(X_train_noisy, 0, 1, out=X_train_noisy)
numpy.clip(X_test_noisy, 0, 1, out=X_test_noisy)

In [ ]:
n = 10
f, ax = plt.subplots(1, n, figsize=(n * 1.4, 2))
for i in range(n):
    ax[i].imshow(X_test_noisy[i].reshape(28, 28), cmap="gray_r")
    ax[i].set_title(y_test[i])
    ax[i].axis("off")
plt.show()

## Création de l'autoencodeur débruiteur



In [ ]:
# Votre code ici
encoding_dim = 4
model = keras.models.Sequential()

### Solution

In [ ]:
encoding_dim = 4
model = keras.models.Sequential()
model.add(keras.layers.Input(shape=input_dim))
model.add(keras.layers.Dense(encoding_dim, activation="relu"))
model.add(keras.layers.Dense(input_dim, activation="sigmoid"))
model.compile(optimizer="adam", loss="mean_squared_error")

## Apprentissage

*Écrivez la ligne correspondant à l'apprentissage de votre autoencodeur :*

- *50 itérations devraient suffire*
- *Utilisez un batch de 256*

In [ ]:
# Votre code ici

##Solution

In [ ]:
model.fit(X_train_noisy, X_train,
          epochs=50,
          batch_size=256,
          validation_split=0.2)

## Base de Test

Autoencodez les images de test et stockez les images obtenues dans la variable `decoded_imgs`

In [ ]:
# Votre code ici
decoded_imgs = X_test_noisy

### Solution

In [ ]:
decoded_imgs = model.predict(X_test_noisy)

## Affichage visuel de la performance

In [ ]:
n = 10
f, ax = plt.subplots(2, n, figsize=(n * 1.4, 4))
for i in range(n):
    # L'original en haut
    ax[0, i].imshow(X_test_noisy[i].reshape(28, 28), cmap="gray_r")
    ax[0, i].set_title(str(y_test[i]))
    ax[0, i].axis("off")

    # La reconstruction en bas
    ax[1, i].imshow(decoded_imgs[i].reshape(28, 28), cmap="gray_r")
    ax[1, i].axis("off")
plt.show()

## Essayez avec plus de neurones

Que se passe-t-il ?

## Utilisation des réseaux convolutifs

Pour cela il faut remettre chaque image sous forme 28x28x1. Les CNNs ont besoin de cette 3ème dimension de tenseur (il pourrait y avoir plus de canaux que le niveau de gris : il y en a 3 pour les images en couleur et plus encore dans les couches intermédiaires d'un réseau convolutif où le nombre de canaux en entrée d'une couche sera le nombre de kernels de la couche précédente).

In [ ]:
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)
X_train_noisy = X_train_noisy.reshape(-1, 28, 28, 1)
X_test_noisy = X_test_noisy.reshape(-1, 28, 28, 1)

## Création du modèle

On utilisera des séquences de Conv2D,MaxPool2D avec des kernel de respectivement 3x3 et 2x2 pour la partie encodeur. Dans ces deux layers, il faudra utiliser l'option "padding='same'" afin d'éviter les effets de bord (l'image étant déjà assez petite comme ça).

Pour la partie decodeur, on utilisera des Conv2D de même nature suivis par des UpSampling2D((2,2)) qui correspond à l'opération 'inverse' de MaxPool2D

N'hésitez pas à abuser de model.summary() pour vous y retrouver. L'objectif étant de retrouver une image 28x28x1 à la sortie du dernier Conv2D. En effet, finir par un Upsampling2D serait une mauvaise idée.

Le réseau va être profond, on utilisera des fonctions d'activation relu, sauf pour la dernière où on utilisera une sigmoïde.

In [ ]:
# Votre code ici
model = keras.models.Sequential()

### Solution

In [ ]:
model = keras.models.Sequential()
# Couches d'encodage
model.add(keras.layers.Input(shape=X_train.shape[1:]))
model.add(keras.layers.Conv2D(32, (3, 3), activation="relu", padding="same"))
model.add(keras.layers.MaxPool2D((2, 2), padding="same"))
model.add(keras.layers.Conv2D(32, (3, 3), activation="relu", padding="same"))
model.add(keras.layers.MaxPool2D((2, 2), padding="same"))
model.add(keras.layers.Conv2D(1, (3, 3), activation="relu", padding="same"))

# Couches de décodage
model.add(keras.layers.Conv2D(32, (3, 3), activation="relu", padding="same"))
model.add(keras.layers.UpSampling2D((2, 2)))
model.add(keras.layers.Conv2D(32, (3, 3), activation="relu", padding="same"))
model.add(keras.layers.UpSampling2D((2, 2)))
model.add(keras.layers.Conv2D(1, (3, 3), activation="sigmoid", padding="same"))

model.compile(optimizer="adam", loss="binary_crossentropy")

model.summary()

## Apprentissage

*Écrivez la ligne correspondant à l'apprentissage de votre autoencodeur :*

- *100 itérations devraient suffire*
- *Utilisez un batch de 256*

In [ ]:
# Votre code ici

### Solution

In [ ]:
model.fit(X_train_noisy, X_train,
          epochs=100,
          batch_size=128,
          validation_split=0.2)

## Affichage des performances

In [ ]:
decoded_imgs = model.predict(X_test_noisy).reshape(-1, 28, 28)

n = 10
f, ax = plt.subplots(2, n, figsize=(n * 1.4, 4))

random_indexes = numpy.random.choice(X_test.shape[0],
                                     replace=False,
                                     size=n)

for i, random_index in enumerate(random_indexes):
    # L'image originale en haut
    ax[0, i].set_title(str(y_test[random_index]))
    ax[0, i].imshow(X_test_noisy[random_index].reshape(28, 28), cmap="gray_r")
    ax[0, i].axis("off")

    # L'image reconstruite en bas
    ax[1, i].imshow(decoded_imgs[random_index], cmap="gray_r")
    ax[1, i].axis("off")
plt.show()

## Sur des images non-bruitées

In [ ]:
decoded_imgs = model.predict(X_test).reshape(-1, 28, 28)

n = 10
f, ax = plt.subplots(2, n, figsize=(n * 1.4, 4))

random_indexes = numpy.random.choice(X_test.shape[0],
                                     replace=False,
                                     size=n)

for i, random_index in enumerate(random_indexes):
    # L'image originale en haut
    ax[0, i].set_title(str(y_test[random_index]))
    ax[0, i].imshow(X_test[random_index].reshape(28, 28), cmap="gray_r")
    ax[0, i].axis("off")

    # L'image reconstruite en bas
    ax[1, i].imshow(decoded_imgs[random_index], cmap="gray_r")
    ax[1, i].axis("off")
plt.show()